Ben: A lot of unfamiliar packages here, more on the way. pipenv/virtual environments are your friend. A quick disclaimer that I am barely familiar with this domain of financial modeling, i can be very wrong in my understanding of things. i hope my code is readable.

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import sleep

from pandas_datareader import data as pdr
# import pandas_ta as ta #commented out for me for now, I (Ben) had some dependency issues
import yfinance as yf
# import quandl as qd # not used, ignor ples
from backtesting import Backtest, Strategy

In [3]:
yf.pdr_override() # for use with pandas-datareader, optional

# Data Sources/Exploration
## which stocks/tickers will we be looking at?
We've more or less arbitrarily settled on the current S&P 500 index, a popular choice.
## What data do we need, and where can we get it?
We need OHLC (open high low close) data, on a monthly and daily scale. Using yfinance, a module that basically scrapes the ya"hoo finance site for data on tickers, we can get most of what we need.
## 

In [26]:
ticker = pd.read_csv('ticker.csv')['Ticker']
tickers = ticker.to_list() # This is a list of all tickers in the SP500
tickers = [x.replace('.','-') for x in tickers] # yahoo has '-' instead of '.' for tickers, eg BRK.B
ticker.head()

0     AAPL
1     MSFT
2     AMZN
3       FB
4    GOOGL
Name: Ticker, dtype: object

In [27]:
# sp_500 = {x:yf.Ticker(x) for x in tickers} # a dict of initialized yfinance tickers
# sp_500['AAPL'].history(start='2019-01-01', end='2020-12-31', interval='1mo')
# sp_500['AAPL'].history(start='2010-01-01', end='2020-12-31')

In [65]:
## below is how I got monthly and daily pandas dataframes of all stocks in one huge dict.  
monthly_sp500 = {}
# for tkr in tickers: # run all 500 at your own risk, it takes a while lol
for tkr in tickers[:10]:
    print('Grabbing ' + tkr + " data!")
    # monthly_sp500[tkr] = yf.Ticker(tkr).history(start='2010-01-01', end='2020-12-31', interval='1mo')
    monthly_sp500[tkr] = pdr.get_data_yahoo(ticker, start="2010-01-04", interval = "1mo")
    sleep(.1) # not planning on ddos-ing yahoo today
monthly_sp500['AAPL'] # take AAPL, for example; TODO deal with the pesky NaNs properly

ata!
[*********************100%***********************]  1 of 1 completed
Grabbing AJG data!
[*********************100%***********************]  1 of 1 completed
Grabbing FLT data!
[*********************100%***********************]  1 of 1 completed
Grabbing TTWO data!
[*********************100%***********************]  1 of 1 completed
Grabbing HSY data!
[*********************100%***********************]  1 of 1 completed
Grabbing CERN data!
[*********************100%***********************]  1 of 1 completed
Grabbing CDW data!
[*********************100%***********************]  1 of 1 completed
Grabbing FTNT data!
[*********************100%***********************]  1 of 1 completed
Grabbing OXY data!
[*********************100%***********************]  1 of 1 completed
Grabbing VTRS data!
[*********************100%***********************]  1 of 1 completed
Grabbing WST data!
[*********************100%***********************]  1 of 1 completed
Grabbing O data!
[*********************100

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-02-01,63.259998,64.699997,61.889999,63.000000,45.095860,213341500.0
2010-02-19,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-01,63.270000,65.489998,63.020000,65.199997,47.027702,249947500.0
2010-04-01,65.370003,66.199997,64.209999,64.300003,46.378559,235860700.0
2010-05-01,65.160004,65.620003,58.189999,58.299999,42.050858,357775800.0
...,...,...,...,...,...,...
2020-12-01,146.289993,157.660004,145.860001,157.380005,156.404709,158930200.0
2021-01-01,157.240005,173.649994,154.130005,163.130005,162.119080,184458400.0
2021-02-01,165.309998,167.940002,159.570007,162.759995,161.751358,133198000.0


In [68]:
daily_sp500 = {}
# for tkr in tickers:
for tkr in tickers[:10]: #this only gets the first 10 fyi
    print('Grabbing ' + tkr + " data!")
    # daily_sp500[tkr] = yf.Ticker(tkr).history(start='2010-01-01', end='2020-12-31', interval='1mo')
    daily_sp500[tkr] = pdr.get_data_yahoo(tkr,start="2010-01-01")
    sleep(.1)
daily_sp500['AAPL'] # take AAPL, for example

ata!
[*********************100%***********************]  1 of 1 completed
Grabbing AJG data!
[*********************100%***********************]  1 of 1 completed
Grabbing FLT data!
[*********************100%***********************]  1 of 1 completed
Grabbing TTWO data!
[*********************100%***********************]  1 of 1 completed
Grabbing HSY data!
[*********************100%***********************]  1 of 1 completed
Grabbing CERN data!
[*********************100%***********************]  1 of 1 completed
Grabbing CDW data!
[*********************100%***********************]  1 of 1 completed
Grabbing FTNT data!
[*********************100%***********************]  1 of 1 completed
Grabbing OXY data!
[*********************100%***********************]  1 of 1 completed
Grabbing VTRS data!
[*********************100%***********************]  1 of 1 completed
Grabbing WST data!
[*********************100%***********************]  1 of 1 completed
Grabbing O data!
[*********************100

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,7.622500,7.660714,7.585000,7.643214,6.583586,493729600
2010-01-05,7.664286,7.699643,7.616071,7.656429,6.594968,601904800
2010-01-06,7.656429,7.686786,7.526786,7.534643,6.490066,552160000
2010-01-07,7.562500,7.571429,7.466071,7.520714,6.478067,477131200
2010-01-08,7.510714,7.571429,7.466429,7.570714,6.521136,447610800
...,...,...,...,...,...,...
2021-02-22,128.009995,129.720001,125.599998,126.000000,126.000000,103607600
2021-02-23,123.760002,126.709999,118.389999,125.860001,125.860001,157859100
2021-02-24,124.940002,125.559998,122.230003,125.349998,125.349998,110691500


In [75]:
# everything in one stupid large dataframe
sp500m = monthly_sp500[tickers[0]] # monthly
sp500m['Name'] = tickers[0]

sp500d = daily_sp500[tickers[0]] # daily
sp500d['Name'] = tickers[0]

# for tkr in tickers[1:10]:
for tkr in tickers:
    df1 = monthly_sp500[tkr] # monthly
    df1['Name'] = tkr
    # df.set_index(['Name','Date'],inplace=True)
    sp500m = sp500_m.append(df1)

    df2 = daily_sp500[tkr] # daily
    df2['Name'] = tkr
    # df.set_index(['Name','Date'],inplace=True)
    sp500d = sp500_d.append(df2)
sp500m

,Open,High,Low,Close,Adj Close,Volume,Name
Date,,,,,,,
2010-02-01,63.259998,64.699997,61.889999,63.000000,45.095860,213341500.0,AAPL
2010-02-19,NaN,NaN,NaN,NaN,NaN,NaN,AAPL
2010-03-01,63.270000,65.489998,63.020000,65.199997,47.027702,249947500.0,AAPL
2010-04-01,65.370003,66.199997,64.209999,64.300003,46.378559,235860700.0,AAPL
2010-05-01,65.160004,65.620003,58.189999,58.299999,42.050858,357775800.0,AAPL
...,...,...,...,...,...,...,...
2020-12-01,146.289993,157.660004,145.860001,157.380005,156.404709,158930200.0,NWS
2021-01-01,157.240005,173.649994,154.130005,163.130005,162.119080,184458400.0,NWS
2021-02-01,165.309998,167.940002,159.570007,162.759995,161.751358,133198000.0,NWS


In [ ]:
sp500m

In [78]:
print(sp500m.shape,sp500d.shape)
sp500d

(90753, 7) (1363416, 7)


,Open,High,Low,Close,Adj Close,Volume,Name
Date,,,,,,,
2010-01-04,7.622500,7.660714,7.585000,7.643214,6.583586,493729600.0,AAPL
2010-01-05,7.664286,7.699643,7.616071,7.656429,6.594968,601904800.0,AAPL
2010-01-06,7.656429,7.686786,7.526786,7.534643,6.490066,552160000.0,AAPL
2010-01-07,7.562500,7.571429,7.466071,7.520714,6.478067,477131200.0,AAPL
2010-01-08,7.510714,7.571429,7.466429,7.570714,6.521136,447610800.0,AAPL
...,...,...,...,...,...,...,...
2021-02-22,22.500000,23.330000,22.370001,23.110001,23.110001,362000.0,NWS
2021-02-23,22.969999,23.180000,22.709999,23.080000,23.080000,497600.0,NWS
2021-02-24,23.040001,23.600000,22.930000,23.240000,23.240000,465800.0,NWS


In [79]:
# sp500m.to_csv('SP500_monthly.csv')
# sp500d.to_csv('SP500_daily.csv')
# pd.read_csv('SP500_monthly.csv').head()

## ----prototyping below----

In [58]:
aapl = monthly_sp500['AAPL'] # take a look at apple again, this time with no nulls (dont know how to deal with them help)
aapl.dropna()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-02-01,63.259998,64.699997,61.889999,63.000000,45.095860,213341500.0
2010-03-01,63.270000,65.489998,63.020000,65.199997,47.027702,249947500.0
2010-04-01,65.370003,66.199997,64.209999,64.300003,46.378559,235860700.0
2010-05-01,65.160004,65.620003,58.189999,58.299999,42.050858,357775800.0
2010-06-01,58.209999,60.150002,57.549999,59.060001,42.988129,357343400.0
...,...,...,...,...,...,...
2020-11-01,138.979996,151.300003,137.490005,144.679993,142.791168,153635100.0
2020-12-01,146.289993,157.660004,145.860001,157.380005,156.404709,158930200.0
2021-01-01,157.240005,173.649994,154.130005,163.130005,162.119080,184458400.0


In [62]:
## shamelessly stolen code, still trying to comprehend what wizardry is going on in here
# dfs = []
# k = 4
# for ticker in tickers[:10]:
#     print('Grabbing ' + ticker + " data!")
#     curr_df = pdr.get_data_yahoo(ticker, start="2010-01-04", interval = "1mo")
#     if(curr_df.empty):
#         print('Failed to get ' + ticker + ' stock data!')
#         continue
#     curr_df['Name'] = ticker
#     curr_df = curr_df.reset_index()
#     curr_df.set_index(['Name','Date'],inplace=True)
#     curr_df = curr_df[~curr_df.index.duplicated(keep='first')]
#     obs_df = (curr_df.reset_index(level=[0])
#         ['Adj Close'].resample(str(k) + 'M')
#         .ffill().pct_change()[1:]).to_frame()
#     obs_df['Name'] = ticker
#     obs_df = obs_df.reset_index()
#     obs_df.set_index(['Name','Date'],inplace=True)
#     obs_df['Cum Returns'] = (obs_df['Adj Close'] + 1).groupby('Name').cumprod()
#     dfs.append(obs_df)
#     curr_df = None
#     obs_df = None

In [47]:
aapl

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-02-01,63.259998,64.699997,61.889999,63.000000,45.095860,213341500.0
2010-02-19,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-01,63.270000,65.489998,63.020000,65.199997,47.027702,249947500.0
2010-04-01,65.370003,66.199997,64.209999,64.300003,46.378559,235860700.0
2010-05-01,65.160004,65.620003,58.189999,58.299999,42.050858,357775800.0
...,...,...,...,...,...,...
2020-12-01,146.289993,157.660004,145.860001,157.380005,156.404709,158930200.0
2021-01-01,157.240005,173.649994,154.130005,163.130005,162.119080,184458400.0
2021-02-01,165.309998,167.940002,159.570007,162.759995,161.751358,133198000.0


# Modeling:
Below we start to really look at feature engineering, not much here yet

In [60]:
def features(ticker_df):
    #
    returns  = ticker_df['Adj Close'].resample('M').ffill().pct_change() 
    cum_returns = (returns + 1).cumprod()
    cum_returns

In [61]:
# pd.read_csv('constituent_performance_4_months.csv')

In [ ]:
# momentum/ model: given a huge dataset with daily/monthly data, we determine in an initial period J to find which tickers do the best/worst, 

## ---notes below---

# For the momentum analysis portion:
The way we plan to proceed for now is to give each stock in our index (sp500) equal weights, in a sense, then rank them by some criteria on how well they are performing. we will adopt the observe J and hold K strategy as detailed in the paper, in which there is an observatory period of J quarters/months/days and a holding period of K quarters/months/days (Ben: idk, j might be able to overlap with k, we can do a lot with how we implement it). In short, we classify in the J period whether a particular ticker would be best treated with a momentum or reversal strategy, or some other. The classification task here is thus presented.

momentum strategy: buy/give heavier weights over time to well performing stocks, as we assume they will continue to get better as per our understanding of momentum

reversal strategy: Amir proposed we short underperforming stocks. This strategy we adopt *might* actually be different from what the paper is proposing, which is to buy and hold underperforming stocks as we predict it to rebound (Ben: fact check me pls).

stocks in between: dont buy them, or leave them as they are...? feel free to share your own thoughts.

# For the modeling portion: 
a lot still left untouched, ill think a bit harder about it tmr ...
we want 3 models to do this kind of momentum analysis algorithmically vs manually, and pit it against a baseline of a simple buy and hold strategy. The parameters will come from a wealth of technical analysis (read: all the fancy hard to understand financial forecasting stuff) libraries already available (we probably wont be making most of them, beyond the simple stuff) but we do have a deadline to keep in mind.